In [ ]:
import os
import numpy as np
from PIL import Image, ImageFile
from skimage import io
import matplotlib.pyplot as plt
import tifffile
from scipy import misc
from tifffile import TiffFile
from tifffile.tifffile import TIFF_DECOMPESSORS
from visualization import ids_to_greyscale, load_depth, show_bounding_boxes, load_stencil_ids, load_stencil_flags, \
get_bounding_boxes, show_loaded_bounding_boxes, get_detections
from math import *
from gta_math import *
import pickle

In [ ]:
import visualization
visualization.multi_page = False
visualization.ini_file = 'gta-postprocessing.local.ini'

scene_condition = """
    ( \
        SELECT scene_id \
        FROM snapshots \
        WHERE run_id = 6 \
        ORDER BY timestamp DESC \
        OFFSET 80 \
        LIMIT 1 \
    ) \
    """

scene_condition = """
    '1ca7f4b8-ef30-4a9f-8657-bc977dd04a89' \
    """

conn = visualization.get_connection()
cur = conn.cursor()
cur.execute("""SELECT snapshot_id, imagepath, cam_near_clip, cam_far_clip, timestamp, view_matrix, proj_matrix, world_matrix, 
    width, height, \
    ARRAY[st_x(camera_relative_rotation), st_y(camera_relative_rotation), st_z(camera_relative_rotation)] as relative_cam_rot, \
    ARRAY[st_x(camera_pos), st_y(camera_pos), st_z(camera_pos)] as camera_pos, \
    ARRAY[st_x(camera_rot), st_y(camera_rot), st_z(camera_rot)] as camera_rot, \
    ARRAY[st_x(camera_direction), st_y(camera_direction), st_z(camera_direction)] as camera_direction \
    FROM snapshots \
    WHERE scene_id = {} \
    ORDER BY snapshot_id ASC \
    """.format(scene_condition))

results = [dict(res) for res in cur]
for i, res in enumerate(results):
    res['view_matrix'] = np.array(res['view_matrix'], dtype=np.float64)
    res['world_matrix'] = np.array(res['world_matrix'], dtype=np.float64)
    res['proj_matrix'] = np.array(res['proj_matrix'], dtype=np.float64)
    res['relative_cam_rot'] = np.array(res['relative_cam_rot'], dtype=np.float64)
    res['camera_pos'] = np.array(res['camera_pos'], dtype=np.float64)
    res['camera_rot'] = np.array(res['camera_rot'], dtype=np.float64)
    res['camera_direction'] = np.array(res['camera_direction'], dtype=np.float64)
    results[i] = res

names = [i['imagepath'] for i in results]
print(names[0])

for res in results:
    res['detections'] = get_detections(res['imagepath'])

with open('db_data.rick', mode='wb+') as file:
    pickle.dump(results, file)

### loads pickled data instead of database

In [ ]:
with open('db_data.rick', mode='rb') as file:
    results = pickle.load(file)
    print('I\'m pickle Rick!!!')

### obtaining detections for this scene

In [ ]:
detections = {}
for res in results:
    cur.execute("""SELECT detection_id, type, class, bbox, \
        snapshot_id, handle, ARRAY[st_x(pos), st_y(pos), st_z(pos)] AS pos \
        FROM detections \
        WHERE snapshot_id = {} \
        AND type = 'car' \
        AND NOT bbox @> point '(Infinity, Infinity)' \
        """.format(res['snapshot_id']))
    detections[res['snapshot_id']] = [dict(i) for i in cur]


In [ ]:
for res in results:  
    print('detections for image {}:'.format(res['snapshot_id']))
    for det in detections[res['snapshot_id']]:
        print(det)

### showing rgb images

In [ ]:
%matplotlib inline
for name in names:
    im = Image.open(os.path.join(visualization.get_in_directory(), name + '.tiff'))
    fig = plt.figure(figsize=(12,12))
    plt.axis('off')
    plt.imshow(im)
    plt.show()

### showing depths

In [ ]:
%matplotlib inline
for name in names:
    depth = load_depth(name)
    fig = plt.figure(figsize=(12,12))
    plt.axis('off')
    plt.imshow(depth, cmap='gray')
    plt.show()


### showing stencil ids

In [ ]:
%matplotlib inline
for name in names:
    depth = load_stencil_ids(name)
    fig = plt.figure(figsize=(12,12))
    plt.axis('off')
    plt.imshow(depth, cmap='gray')
    plt.show()


### tinkering with depth, showing its values

Depth is in NDC, openGL's Normalized Device Coordinates. 

According to [this thread](https://www.opengl.org/discussion_boards/showthread.php/170718-Normalized-Device-Coordinates) 
it maps from $[near clip (n_c), far clip(f_c)]$ to $[-1, 1]$.

So the formula for calculating NDC from world coordinates (WC) is simple linear transformation:

$ NDC = k \cdot WC + q $

$ -1 = k \cdot n_c + q $

$ 1 = k \cdot f_c + q $

After solving these equations, we obtain transformation parameters

$ q = -1 - k \cdot n_c $

$ q = 1 - k \cdot f_c $

$ -1 - k \cdot n_c = 1 - k \cdot f_c $

$ - k \cdot n_c = 2 - k \cdot f_c $

$ -2 = k (n_c - f_c) $

$ k = \frac{-2}{n_c - f_c} $

$ q = 1 - k \cdot f_c $

$ q = 1 - \frac{-2}{n_c - f_c} \cdot f_c $

$ q = 1 + \frac{2 f_c}{n_c - f_c} $

So the resulting linear transformations is as follows

$ NDC = \frac{-2}{n_c - f_c} \cdot WC + \frac{2 f_c}{n_c - f_c} $

And the reverse transformation, from NDC back to WC, shall be obtained by the same approach

$ WC = k \cdot NDC + q $

$ n_c = k \cdot -1 + q $

$ f_c = k \cdot 1 + q $

$ n_c = -k + q $

$ f_c = k + q $

After solving these equations, we obtain transformation parameters

$ q = n_c + k $

$ q = f_c - k $

$ n_c + k = f_c - k $

$ n_c + 2k = f_c $

$ k = \frac{f_c - n_c}{2} $

$ q = f_c - k $

$ q = f_c - \frac{f_c - n_c}{2} $

$ q = f_c + \frac{-f_c + n_c}{2} $

$ q = \frac{f_c + n_c}{2} $

And resulting transformation is

$ WC = \frac{f_c - n_c}{2} \cdot NDC + \frac{f_c + n_c}{2} $



For mapping from and to $[0, 1]$

So the formula for calculating NDC from world coordinates (WC) is simple linear transformation:

$ NDC = k \cdot WC + q $

$ 0 = k \cdot n_c + q $

$ 1 = k \cdot f_c + q $

After solving these equations, we obtain transformation parameters

$ q = 0 - k \cdot n_c $

$ q = 1 - k \cdot f_c $

$ 0 - k \cdot n_c = 1 - k \cdot f_c $

$ - k \cdot n_c = 1 - k \cdot f_c $

$ -1 = k (n_c - f_c) $

$ k = \frac{-1}{n_c - f_c} $

$ q = - k \cdot f_c $

$ q = - \frac{-1}{n_c - f_c} \cdot f_c $

$ q = \frac{f_c}{n_c - f_c} $

So the resulting linear transformations is as follows

$ NDC = \frac{-1}{n_c - f_c} \cdot WC + \frac{f_c}{n_c - f_c} $

And the reverse transformation, from NDC back to WC, shall be obtained by the same approach

$ WC = k \cdot NDC + q $

$ n_c = k \cdot 0 + q $

$ f_c = k \cdot 1 + q $

$ n_c = q $

$ f_c = k + q $

After solving these equations, we obtain transformation parameters

$ q = n_c $

$ q = f_c - k $

$ n_c = f_c - k $

$ n_c + k = f_c $

$ k = f_c - n_c $

$ q = f_c - k $

$ q = f_c - (f_c - n_c) $

$ q = n_c $

And resulting transformation is

$ WC = (f_c - n_c) \cdot NDC + n_c $


In [ ]:
for res in results:    
    name = res['imagepath']
    depth = load_depth(name)
    near_clip = res['cam_near_clip']
    far_clip = res['cam_far_clip']
    print('camera rot: {}'.format(res['relative_cam_rot']))
    print('image name: {}'.format(res['imagepath']))
    print('created: {}'.format(res['timestamp']))
    print('depth min: {}'.format(np.min(depth)))
    print('depth max: {}'.format(np.max(depth)))
    print('near clip: {}'.format(near_clip))
    print('far clip: {}'.format(far_clip))
    
    k = (far_clip - near_clip) / 2
    q = (far_clip + near_clip) / 2
    new_depth = depth * k + q
    print('new depth min: {}'.format(np.min(new_depth)))
    print('new depth max: {}'.format(np.max(new_depth)))
    
    # fig = plt.figure(figsize=(12,12))
    # plt.axis('off')
    # plt.imshow(new_depth, cmap='gray')
    # plt.show()


### showing bounding boxes

In [ ]:
name = names[3]
im = Image.open(os.path.join(visualization.get_in_directory(), name + '.tiff'))
fig = plt.figure(figsize=(12,12))
plt.axis('off')
plt.imshow(im)
size = (im.size[1], im.size[0])
show_bounding_boxes(name, size, plt.gca())


### showing bounding boxes for all cameras

In [ ]:
for name in names:
    im = Image.open(os.path.join(visualization.get_in_directory(), name + '.tiff'))
    fig = plt.figure(figsize=(12,12))
    plt.axis('off')
    plt.imshow(im)
    size = (im.size[1], im.size[0])
    show_bounding_boxes(name, size, plt.gca())


In [ ]:
name = names[2]
im = Image.open(os.path.join(visualization.get_in_directory(), name + '.tiff'))
fig = plt.figure(figsize=(12,12))
plt.axis('off')
plt.imshow(im)
size = (im.size[1], im.size[0])
detections = get_bounding_boxes(name)
detections = [d for d in detections if (d['bbox'][0, 0] - d['bbox'][1, 0]) > 0.2]
show_loaded_bounding_boxes(detections, size, plt.gca())
for det in detections:
    print('pos: {}, type: {}, class: {}, handle: {}'.format(det['pos'], det['type'], det['class'], det['handle']))

In [ ]:
print('columns: '+', '.join(detections[0].keys()))
for det in detections:
    bbox = det['bbox']
    print(bbox)
    print(det['bbox'][0, 0] - det['bbox'][1, 0])
    print(det['pos'])

#detections = get_detections(name) ## this loads them from db, do not run
detections = res['detections']
# car 0 je to černé
car_0 = [d for d in detections if d['handle'] == 53250][0]
# car 1 je to bílé
car_1 = [d for d in detections if d['handle'] == 52994][0]

print('car_0 pos: '+ str(car_0['pos']))
print('car_1 pos: '+ str(car_1['pos']))
print('cars distance: '+ str(np.linalg.norm(car_0['pos'][0:2] - car_1['pos'][0:2])))


### playing with depth and bounding boxes

In [ ]:
name = names[3]
im = Image.open(os.path.join(visualization.get_in_directory(), name + '.tiff'))
fig = plt.figure(figsize=(12,12))
plt.axis('off')
plt.imshow(im)
size = (im.size[1], im.size[0])
detections = get_bounding_boxes(name)
print('detections before filtering: ' + str(len(detections)))
# sizes = [str(d['bbox'][0, 0] - d['bbox'][1, 0]) for d in detections]
# print('sizes: '+', '.join(sizes))
orig_detections = detections
detections = [d for d in detections if (d['bbox'][0, 0] - d['bbox'][1, 0]) > 0.2]
show_loaded_bounding_boxes(detections, size, plt.gca())
print('detections after filtering: ' + str(len(detections)))

depth = load_depth(name)
fig = plt.figure(figsize=(12,12))
plt.axis('on')
# plt.gca().grid(color='r', linestyle='-', linewidth=1)
point_0 = (400, 700)
point_1 = (400, 900)
plt.plot([point_0[1], point_1[1]], [point_0[0], point_1[0]], 'o')
plt.imshow(depth, cmap='gray')
plt.show()

In [ ]:
print('raw point 0: {}'.format(depth[point_0]))
print('raw point 1: {}'.format(depth[point_1]))
print('raw depth distance: '+ str(depth[point_1] - depth[point_0]))

k = (far_clip - near_clip) / 2
q = (far_clip + near_clip) / 2
new_depth = depth * k + q
print('-1,1 depth distance: {}'.format(new_depth[point_1] - new_depth[point_0]))

k = far_clip - near_clip
q = near_clip
new_depth = depth * k + q
print('0, 1 point 0: {}'.format(new_depth[point_0]))
print('0, 1 point 1: {}'.format(new_depth[point_1]))
print('0, 1 depth distance: {}'.format(new_depth[point_1] - new_depth[point_0]))

c = 1
new_depth = (np.exp(depth * np.log(far_clip * c + 1)) - 1)/c
print('exp depth distance: {}'.format(new_depth[point_1] - new_depth[point_0]))
new_depth = 1 / new_depth
print('exp invered point 0: {}'.format(new_depth[point_0]))
print('exp invered point 1: {}'.format(new_depth[point_1]))
print('exp invered depth distance: {}'.format(new_depth[point_1] - new_depth[point_0]))

res = results[3]
print('keys: '+', '.join(res.keys()))
print('view matrix:')
print(np.array(res['view_matrix']))
print('proj matrix:')
print(np.array(res['proj_matrix']))


In [ ]:
depth[899, 800]

In [ ]:
depth[100, 1400]

In [ ]:
print(np.min(depth))
print(np.max(depth))

In [ ]:
print(np.min(new_depth))
print(np.max(new_depth))
print(depth.shape)

In [ ]:
 vec = (700/depth.shape[1], 500/depth.shape[0], depth[500, 700])

In [ ]:
submatrix = res['view_matrix'][0:3, 0:3]
print(submatrix)
np.linalg.det(submatrix)


In [ ]:
proj_matrix = res['proj_matrix']
vec = np.linalg.inv(proj_matrix) @ np.array([[1], [1], [0.002], [1]])
print(vec / vec[3])
print(res['cam_near_clip'])
print(res['cam_far_clip'])

print('proj_matrix')
print(proj_matrix)
print('inverted proj_matrix')
print(np.linalg.inv(proj_matrix))

print('rounding small values to zero')
proj_matrix[np.abs(proj_matrix) < 1e-7] = 0

print('proj_matrix')
print(proj_matrix)
print('inverted proj_matrix')
print(np.linalg.inv(proj_matrix))


el1 = (- far_clip - near_clip)/(2*(near_clip*far_clip))
el2 = (far_clip + near_clip)/(2*(near_clip*far_clip))
print(el1)
print(el2)

matrix = np.zeros((4, 4))
matrix[2, 3] = -1
matrix[3, 2] = el1
matrix[3, 3] = el2
print(matrix)
vec = matrix @ np.array([[1], [1], [0], [1]])
print(vec / vec[3])
print(proj_matrix[0,0])
print(1/ np.linalg.inv(proj_matrix)[0,0])

In [ ]:
vec = np.array([1, 1, 1, 1]) @ np.linalg.inv(np.array(res['proj_matrix']))
vec / vec[3]

In [ ]:
print(res['cam_near_clip'])
print(res['cam_far_clip'])

print('transforming near clip')
proj_matrix = res['proj_matrix']
vec = proj_matrix @ np.array([[1], [1], [-res['cam_near_clip']], [1]])
print(vec)
print(vec / vec[3])

print('transforming far clip')
proj_matrix = res['proj_matrix']
vec = proj_matrix @ np.array([[1], [1], [-res['cam_far_clip']], [1]])
print(vec)
print(vec / vec[3])


### preparing correct form of coordinates for inverse projection

In [ ]:
width = res['width']
height = res['height']
size = (height, width)
near_clip = res['cam_near_clip']
far_clip = res['cam_far_clip']

# last column form
ndc_0 = pixel_to_normalized(point_0, size)
ndc_1 = pixel_to_normalized(point_1, size)
vec_0 = [ndc_0[1], ndc_0[0], -depth[point_0], 1]
vec_1 = [ndc_1[1], ndc_1[0], -depth[point_1], 1]
vec_0 = np.array(vec_0)[:, np.newaxis]
vec_1 = np.array(vec_1)[:, np.newaxis]

print('constructed points')
print(vec_0)
print(vec_1)

vec_0_p = np.linalg.inv(proj_matrix) @ vec_0
vec_1_p = np.linalg.inv(proj_matrix) @ vec_1

print('projected by inversion')
print(vec_0_p)
print(vec_1_p)

print('normalized')
vec_0_p /= vec_0_p[3]
vec_1_p /= vec_1_p[3]
print(vec_0_p)
print(vec_1_p)

print('distance')
print(np.linalg.norm(vec_0_p - vec_1_p))

In [ ]:
result = np.linalg.inv(proj_matrix) @ np.array([[1], [1], [0.172532], [1]])
result /= result[3]
print(result)

### transformation for two points at once

In [ ]:
point_0 = (400, 700)
point_1 = (400, 900)
width = res['width']
height = res['height']
size = (height, width)
near_clip = res['cam_near_clip']
far_clip = res['cam_far_clip']

# last column form
ndc_0 = pixel_to_normalized(point_0, size)
ndc_1 = pixel_to_normalized(point_1, size)
vec_0 = [ndc_0[1], ndc_0[0], -depth[point_0], 1]
vec_1 = [ndc_1[1], ndc_1[0], -depth[point_1], 1]
vec_0 = np.array(vec_0)[:, np.newaxis]
vec_1 = np.array(vec_1)[:, np.newaxis]

print('constructed points')
print(vec_0)
print(vec_1)

vecs = np.concatenate((vec_0, vec_1), axis=1)
print('merged')
print(vecs)

proj_matrix = res['proj_matrix']
vecs_p = np.linalg.inv(proj_matrix) @ vecs

print('projected by inversion')
print(vecs_p)

print('normalized')
vecs_p /= vecs_p[3, :]
print(vecs_p)

print('points')
vec_0_p = vecs_p[:, 0] 
vec_1_p = vecs_p[:, 1]
print(vec_0_p)
print(vec_1_p)

print('distance')
print(np.linalg.norm(vec_0_p - vec_1_p))

view_matrix = res['view_matrix']
vecs_p = np.linalg.inv(view_matrix) @ vecs_p
vecs_p /= vecs_p[3, :]

print('projected by view')
print(vecs_p)

print('points')
vec_0_p = vecs_p[:, 0] 
vec_1_p = vecs_p[:, 1]
print(vec_0_p)
print(vec_1_p)

print('distance')
print(np.linalg.norm(vec_0_p - vec_1_p))


### showing depth data histogram

In [ ]:
fig = plt.figure()
depth_to_hist = depth.flatten()
print(depth_to_hist.shape)
#plt.hist()
plt.hist(np.random.choice(depth_to_hist, 400000), bins=400)
plt.show()

### calculating zero and non-zero values

In [ ]:
print('depth shape:')
print(depth.shape)
print('depth size:')
print(depth.size)
print('zero values:')
print(depth.size - np.count_nonzero(depth))
print('non-zero values:')
print(np.count_nonzero(depth))

print(depth[0:10, 0:10])

print(depth[0, 2] == 0)

### all pixels to pointcloud

In [ ]:
# all pixels to vectors, or some subset
range_y = range(height)
range_x = range(width)

range_y = range(200, 800)
range_x = range(400, 700)

'''
vecs = np.zeros((4, len(range_y) * len(range_x)))

print(vecs.shape)
i = 0
#for y in range(height):
#    for x in range(width):
for y in range_y:
    for x in range_x:
        ndc = pixel_to_normalized((y, x), size)
        vec = [ndc[1], ndc[0], -depth[(y, x)], 1]
        vec = np.array(vec)
        vecs[:, i] = vec
        i += 1
'''

# only non-zero pixels to pointcloud
vecs = np.zeros((4, np.count_nonzero(depth)))
i = 0
#for y in range(height):
#    for x in range(width):

for y, x in np.transpose(np.nonzero(depth)):
    ndc = pixel_to_normalized((y, x), size)
    vec = [ndc[1], ndc[0], -depth[(y, x)], 1]
    vec = np.array(vec)
    vecs[:, i] = vec
    i += 1


print("prepared, total points: "+str(vecs.shape[1]))

# projection itself
vecs_p = np.linalg.inv(proj_matrix) @ vecs
vecs_p /= vecs_p[3, :]

print("projected")


In [ ]:
%matplotlib notebook
# transformation to pointcloud form
xs = vecs_p[0, ::50]
ys = vecs_p[1, ::50]
zs = vecs_p[2, ::50]

# visualization
from mpl_toolkits.mplot3d import axes3d, Axes3D #<-- Note the capitalization! 
fig = plt.figure()

ax = Axes3D(fig) #<-- Note the difference from your original code...
#ax = fig.add_subplot(111, projection='3d')
ax.scatter(xs, ys, zs, c='b', marker='o')

ax.set_xlabel('X Label')
ax.set_ylabel('Y Label')
ax.set_zlabel('Z Label')

plt.show()


### trying to project cars positions in world coordinates to NDC

In [ ]:
print("cars positions: world")
car_0_pos = car_0['pos']
car_1_pos = car_1['pos']
car_0_pos = np.append(car_0_pos, 1)
car_1_pos = np.append(car_1_pos, 1)
print(car_0_pos)
print(car_1_pos)

print("cars positions: viewed")
view_matrix = res['view_matrix']
car_0_pos = view_matrix @ car_0_pos
car_1_pos = view_matrix @ car_1_pos
car_0_pos /= car_0_pos[3]
car_1_pos /= car_1_pos[3]
print(car_0_pos)
print(car_1_pos)

print("cars positions: projected")
proj_matrix = res['proj_matrix']
car_0_pos = proj_matrix @ car_0_pos
car_1_pos = proj_matrix @ car_1_pos
car_0_pos /= car_0_pos[3]
car_1_pos /= car_1_pos[3]
print(car_0_pos)
print(car_1_pos)

print("cars bboxes")
car_0_bbox = car_0['bbox']
car_1_bbox = car_1['bbox']
print(car_0_bbox)
print(car_1_bbox)


In [ ]:
np.append(car_0_pos, 1)

# projecting multiple points at once and plotting them

#### preparing points

In [ ]:
width = res['width']
height = res['height']
size = (height, width)
near_clip = res['cam_near_clip']
far_clip = res['cam_far_clip']

points = np.array([
    (400, 700),
    (400, 800),
    (400, 900),
    (500, 650),
    (600, 650),
    (700, 650),
    (500, 800),
    (600, 800),
    (700, 800),
])

colors = [
    'r',
    'r',
    'b',
    'b',
    'b',
    'b',
    'b',
    'b',
    'b',
]

y_range = range(0, 900)
x_range = range(0, 1600)
points = np.transpose([np.tile(y_range, len(x_range)), np.repeat(x_range, len(y_range))])


#### plotting them on the depth image

In [ ]:
%matplotlib notebook

import matplotlib as mpl
mpl.rcParams['savefig.dpi'] = 80
mpl.rcParams['figure.dpi'] = 80

name = names[3]
depth = load_depth(name)
fig = plt.figure(figsize=(12,12))
plt.axis('on')
# plt.gca().grid(color='r', linestyle='-', linewidth=1)
#for point, color in zip(points, colors):
#    plt.plot(point[1], point[0], 'o', color=color)
#plt.plot(points[:, 1], points[:, 0], 'o', color=colors)
plt.plot(points[:, 1], points[:, 0], 'o')
plt.imshow(depth, cmap='gray')

#### transforming and projecting them

In [ ]:
# when cutting too far points, I project far clip to depth and then use its value to treshold:
print('transforming far clip')
proj_matrix = res['proj_matrix']
max_depth = res['cam_far_clip']
#max_depth = 60 # just for testing
vec = proj_matrix @ np.array([[1], [1], [-max_depth], [1]])
#print(vec)
vec /= vec[3]
treshold = vec[2]

# vecs = np.zeros((4, points.shape[0]))
vecs = np.zeros((4, len(np.where(depth[points[:, 0], points[:, 1]] > treshold)[0])))  # this one is used when ommiting 0 depth (point behind the far clip)
print("vecs.shape")
print(vecs.shape)
i = 0
arr = points
for y, x in arr:
    if depth[(y, x)] <= treshold:
        continue
    ndc = pixel_to_normalized((y, x), size)
    vec = [ndc[1], ndc[0], -depth[(y, x)], 1]
    vec = np.array(vec)
    vecs[:, i] = vec
    i += 1

vecs_p = np.linalg.inv(proj_matrix) @ vecs

print('projected by inversion')
print(vecs_p)

print('normalized')
vecs_p /= vecs_p[3, :]
print(vecs_p)

#### plotting them projected

In [ ]:
%matplotlib notebook
# transformation to pointcloud form
xs = vecs_p[0, :]
ys = vecs_p[1, :]
zs = vecs_p[2, :]

# visualization
from mpl_toolkits.mplot3d import axes3d, Axes3D #<-- Note the capitalization! ,
fig = plt.figure()

ax = Axes3D(fig)
#ax = fig.add_subplot(111, projection='3d')
#for i in range(vecs_p.shape[1]):
#    ax.scatter(xs[i], ys[i], zs[i], c=colors[i], marker='o')
ax.scatter(xs, ys, zs, c='b', marker='o')

ax.set_xlabel('X Label')
ax.set_ylabel('Y Label')
ax.set_zlabel('Z Label')

plt.show()


#### pickle them for plotting them later and elsewhere

In [ ]:
import pickle
with open('points.rick', mode='wb+') as file:
    struct = {
        'points': points, 
        'vecs_p': vecs_p,
        'colors': colors,
        'name': name
    }
    pickle.dump(struct, file)

In [ ]:
a = np.asarray(vecs_p[0:3, :].T)
np.savetxt("points.csv", a, delimiter=",")

### examining same points in two images

In [ ]:
%matplotlib inline
point_0 = (400, 700)
point_1 = (400, 900)

name = names[3]
res = results[3]
im = Image.open(os.path.join(visualization.get_in_directory(), name + '.tiff'))
fig = plt.figure(figsize=(12,12))
plt.axis('off')
plt.plot([point_0[1], point_1[1]], [point_0[0], point_1[0]], 'o')
plt.imshow(im)

depth = load_depth(name)
fig = plt.figure(figsize=(12,12))
plt.axis('on')
plt.plot([point_0[1], point_1[1]], [point_0[0], point_1[0]], 'o')
# plt.gca().grid(color='r', linestyle='-', linewidth=1)
plt.imshow(depth, cmap='gray')
plt.show()

In [ ]:
width = res['width']
height = res['height']
size = (height, width)
near_clip = res['cam_near_clip']
far_clip = res['cam_far_clip']

# last column form
ndc_0 = pixel_to_normalized(point_0, size)
ndc_1 = pixel_to_normalized(point_1, size)
vec_0 = [ndc_0[1], ndc_0[0], -depth[point_0], 1]
vec_1 = [ndc_1[1], ndc_1[0], -depth[point_1], 1]
vec_0 = np.array(vec_0)[:, np.newaxis]
vec_1 = np.array(vec_1)[:, np.newaxis]

print('constructed points')
print(vec_0)
print(vec_1)

vecs = np.concatenate((vec_0, vec_1), axis=1)
print('merged')
print(vecs)

proj_matrix = res['proj_matrix']
vecs_p = np.linalg.inv(proj_matrix) @ vecs

print('projected by inversion')
print(vecs_p)

print('normalized')
vecs_p /= vecs_p[3, :]
print(vecs_p)

print('points')
vec_0_p = vecs_p[:, 0] 
vec_1_p = vecs_p[:, 1]
print(vec_0_p)
print(vec_1_p)

print('distance')
print(np.linalg.norm(vec_0_p - vec_1_p))

view_matrix = res['view_matrix']
vecs_p = np.linalg.inv(view_matrix) @ vecs_p
vecs_p /= vecs_p[3, :]

print('projected by view')
print(vecs_p)

print('points')
vec_0_p = vecs_p[:, 0] 
vec_1_p = vecs_p[:, 1]
print(vec_0_p)
print(vec_1_p)

print('distance')
print(np.linalg.norm(vec_0_p - vec_1_p))

In [ ]:
%matplotlib inline
point_0 = (400, 650)
point_1 = (430, 1180)
name = names[2]
res = results[2]
im = Image.open(os.path.join(visualization.get_in_directory(), name + '.tiff'))
fig = plt.figure(figsize=(12,12))
plt.plot([point_0[1], point_1[1]], [point_0[0], point_1[0]], 'o')
plt.axis('off')
plt.imshow(im)

depth = load_depth(name)
fig = plt.figure(figsize=(12,12))
plt.axis('on')
# plt.gca().grid(color='r', linestyle='-', linewidth=1)
plt.plot([point_0[1], point_1[1]], [point_0[0], point_1[0]], 'o')
plt.imshow(depth, cmap='gray')
plt.show()

In [ ]:
width = res['width']
height = res['height']
size = (height, width)
near_clip = res['cam_near_clip']
far_clip = res['cam_far_clip']

# last column form
ndc_0 = pixel_to_normalized(point_0, size)
ndc_1 = pixel_to_normalized(point_1, size)
vec_0 = [ndc_0[1], ndc_0[0], -depth[point_0], 1]
vec_1 = [ndc_1[1], ndc_1[0], -depth[point_1], 1]
vec_0 = np.array(vec_0)[:, np.newaxis]
vec_1 = np.array(vec_1)[:, np.newaxis]

print('constructed points')
print(vec_0)
print(vec_1)

vecs = np.concatenate((vec_0, vec_1), axis=1)
print('merged')
print(vecs)

proj_matrix = res['proj_matrix']
vecs_p = np.linalg.inv(proj_matrix) @ vecs

print('projected by inversion')
print(vecs_p)

print('normalized')
vecs_p /= vecs_p[3, :]
print(vecs_p)

print('points')
vec_0_p = vecs_p[:, 0] 
vec_1_p = vecs_p[:, 1]
print(vec_0_p)
print(vec_1_p)

print('distance')
print(np.linalg.norm(vec_0_p - vec_1_p))

view_matrix = res['view_matrix']
vecs_p = np.linalg.inv(view_matrix) @ vecs_p
vecs_p /= vecs_p[3, :]

print('projected by view')
print(vecs_p)

print('points')
vec_0_p = vecs_p[:, 0] 
vec_1_p = vecs_p[:, 1]
print(vec_0_p)
print(vec_1_p)

print('distance')
print(np.linalg.norm(vec_0_p - vec_1_p))

In [ ]:
name = names[3]
res = results[3]
width = res['width']
height = res['height']
size = (height, width)
near_clip = res['cam_near_clip']
far_clip = res['cam_far_clip']

points = np.array([
    [400, 700],
    [400, 900]
])
vecs, _ = points_to_homo(points, res, load_depth(name))

print('constructed points')
print(vecs)

proj_matrix = res['proj_matrix']
vecs_p = np.linalg.inv(proj_matrix) @ vecs
vecs_p /= vecs_p[3, :]

print('projected by inversion')
print(vecs_p)

print('distance')
print(np.linalg.norm(vecs_p[:, 0] - vecs_p[:, 1]))

view_matrix = res['view_matrix']
vecs_p = np.linalg.inv(view_matrix) @ vecs_p
vecs_p /= vecs_p[3, :]

print('projected by view')
print(vecs_p)

print('distance')
print(np.linalg.norm(vecs_p[:, 0] - vecs_p[:, 1]))

print('view matrix')
print(view_matrix)

print()

name = names[2]
res = results[2]
width = res['width']
height = res['height']
size = (height, width)
near_clip = res['cam_near_clip']
far_clip = res['cam_far_clip']

points = np.array([
    [400, 650],
    [430, 1180]
])
vecs, _ = points_to_homo(points, res, load_depth(name))

print('constructed points')
print(vecs)

proj_matrix = res['proj_matrix']
vecs_p = np.linalg.inv(proj_matrix) @ vecs
vecs_p /= vecs_p[3, :]

print('projected by inversion')
print(vecs_p)

print('distance')
print(np.linalg.norm(vecs_p[:, 0] - vecs_p[:, 1]))

view_matrix = res['view_matrix']
vecs_p = np.linalg.inv(view_matrix) @ vecs_p
vecs_p /= vecs_p[3, :]

print('projected by view')
print(vecs_p)

print('distance')
print(np.linalg.norm(vecs_p[:, 0] - vecs_p[:, 1]))

print('view matrix')
print(view_matrix)

### examining one point from 2 cameras

In [ ]:
%matplotlib inline

point2 = (440, 972)
name = names[2]
res = results[2]
im = Image.open(os.path.join(visualization.get_in_directory(), name + '.tiff'))
fig = plt.figure(figsize=(12,12))
plt.plot(point2[1], point2[0], 'o')
plt.axis('off')
plt.imshow(im)

point3 = (422, 435)
name = names[3]
res = results[3]
im = Image.open(os.path.join(visualization.get_in_directory(), name + '.tiff'))
fig = plt.figure(figsize=(12,12))
plt.plot(point3[1], point3[0], 'o')
plt.axis('off')
plt.imshow(im)


In [ ]:
def show_view_matrix(view_matrix):
    print('view matrix')
    print(view_matrix)
    print('view translation')
    print(view_matrix[0:3, 3])
    print('view rotation')
    print(view_matrix[0:3, 0:3])
    print('determinant')
    print(np.linalg.det(view_matrix[0:3, 0:3]))
    print('euler angles')
    print(rotationMatrixToEulerAngles(view_matrix[0:3, 0:3]))

def use_inv_proj(proj_matrix, vecs):
    vecs_p = np.linalg.inv(proj_matrix) @ vecs
    vecs_p /= vecs_p[3, :]
    print('projected by inversion')
    print(vecs_p)
    return vecs_p

def use_inv_view(view_matrix, vecs_p):
    vecs_w = np.linalg.inv(view_matrix) @ vecs_p
    vecs_w /= vecs_p[3, :]
    print('projected by view')
    print(vecs_w)
    show_view_matrix(view_matrix)
    return vecs_w
    
def use_inv_world(world_matrix, vecs_p):
    vecs_w = np.linalg.inv(world_matrix) @ vecs_p
    vecs_w /= vecs_p[3, :]
    print('projected by world')
    print(vecs_w)
    print('world matrix')
    print(world_matrix)
    return vecs_w
    
def is_rotation_matrix(R) :
    Rt = np.transpose(R)
    shouldBeIdentity = np.dot(Rt, R)
    I = np.identity(3, dtype = R.dtype)
    n = np.linalg.norm(I - shouldBeIdentity)
    return n < 1e-6

def create_rot_matrix(euler):
    x = np.radians(euler.x)
    y = np.radians(euler.y)
    z = np.radians(euler.z)
    Rx = np.array([[1, 0, 0],
                   [0, cos(x), -sin(x)],
                   [0, sin(x), cos(x)]], dtype=np.float)
    Ry = np.array([[cos(y), 0, sin(y)],
                   [0, 1, 0],
                   [-sin(y), 0, cos(y)]], dtype=np.float)
    Rz = np.array([[cos(z), -sin(z), 0],
                   [sin(z), cos(z), 0],
                   [0, 0, 1]], dtype=np.float)
    result = Rz @ Ry @ Rx
    return result

def rotationMatrixToEulerAngles(R) :
    assert(is_rotation_matrix(R))
    sy = math.sqrt(R[0,0] * R[0,0] +  R[1,0] * R[1,0])
    singular = sy < 1e-6
    if not singular :
        x = math.atan2(R[2,1] , R[2,2])
        y = math.atan2(-R[2,0], sy)
        z = math.atan2(R[1,0], R[0,0])
    else :
        x = math.atan2(-R[1,2], R[1,1])
        y = math.atan2(-R[2,0], sy)
        z = 0
    return np.array([x, y, z])

def use_inv_view_new(res, vecs_p):
    view_matrix_new = np.zeros((4, 4))
    view_matrix_new[0:3, 3] = res['camera_pos']
    view_matrix_new[0:3, 0:3] = create_rot_matrix(res['camera_direction'])
    view_matrix_new[3, 3] = 1

    vecs_w_new = np.linalg.inv(view_matrix_new) @ vecs_p
    vecs_w_new /= vecs_w_new[3, :]

    print('projected by view new')
    print(vecs_w_new)
    show_view_matrix(view_matrix_new)
    return vecs_w_new
    
name = names[2]
res = results[2]
width = res['width']
height = res['height']
size = (height, width)
near_clip = res['cam_near_clip']
far_clip = res['cam_far_clip']
proj_matrix = res['proj_matrix']
view_matrix = res['view_matrix']
world_matrix = res['world_matrix']

vecs, _ = points_to_homo(np.array([point2]), res, name)

print('constructed points')
print(vecs)

vecs_p = use_inv_proj(proj_matrix, vecs)
vecs_w = use_inv_view(view_matrix, vecs_p)
vecs_ww = use_inv_world(world_matrix, vecs_w)
# vecs_w_new = use_inv_view_new(res, vecs_p)

point2world = vecs_w.T[0:3]
# point2world_new = vecs_w_new.T[0:3]

print()

name = names[3]
res = results[3]
width = res['width']
height = res['height']
size = (height, width)
near_clip = res['cam_near_clip']
far_clip = res['cam_far_clip']
proj_matrix = res['proj_matrix']
view_matrix = res['view_matrix']
world_matrix = res['world_matrix']

vecs, _ = points_to_homo(np.array([point3]), res, name)

print('constructed points')
print(vecs)

vecs_p = use_inv_proj(proj_matrix, vecs)
vecs_w = use_inv_view(view_matrix, vecs_p)
vecs_w = use_inv_view(view_matrix, vecs_p)
vecs_ww = use_inv_world(world_matrix, vecs_w)
# vecs_w_new = use_inv_view_new(res, vecs_p)

point3world = vecs_w.T[0:3]
# point3world_new = vecs_w.T[0:3]

print()

print('distance, should be zero')
print(point2world - point3world)
print(np.linalg.norm(point2world - point3world))

# print('new distance, should be zero')
# print(point2world_new - point3world_new)
# print(np.linalg.norm(point2world_new - point3world_new))

cam2world = results[2]['camera_pos']
cam3world = results[3]['camera_pos']
print('cameras positions')
print(cam2world)
print(cam3world)
print('cameras distance')
print(cam2world - cam3world)
print(np.linalg.norm(cam2world - cam3world))


In [ ]:
res = results[2]
proj_matrix = res['proj_matrix']
view_matrix = res['view_matrix']
world_matrix = res['world_matrix']
print(world_matrix)
print('determinant')
print(np.linalg.det(world_matrix[0:3, 0:3]))
view_2 = view_matrix

res = results[3]
proj_matrix = res['proj_matrix']
view_matrix = res['view_matrix']
world_matrix = res['world_matrix']
print(world_matrix)
print('determinant')
print(np.linalg.det(world_matrix[0:3, 0:3]))
view_3 = view_matrix


In [ ]:
print(view_2)
print(np.linalg.det(view_2[0:3, 0:3]))
print(view_3)
print(np.linalg.det(view_3[0:3, 0:3]))


In [ ]:
r21 = (view_3[0:3, 0:3]) @ (view_2[0:3, 0:3].T)

In [ ]:
t21 = view_3[0:3, 3] - r21 @ view_2[0:3, 3]
print(t21)
print(np.linalg.norm(t21))

print(math.sqrt(2)*8)

In [ ]:
ndc = pixel_to_ndc((800, 1200), (900, 1600))
print(ndc)

In [ ]:
pixel = ndc_to_pixel(ndc, (900, 1600))
print(pixel)

In [ ]:
res = results[0]
view_matrix = res['view_matrix']

In [ ]:
def is_rigid(M):
    return is_rotation_matrix(M[0:3, 0:3]) and np.linalg.norm(M[3, :] - np.array([0, 0, 0, 1], dtype=M.dtype)) < 1e-6


def inv_rigid(M):
    assert is_rigid(M)
    Mt = np.zeros_like(M)
    Mt[0:3, 0:3] = np.transpose(M[0:3, 0:3])
    Mt[0:3, 3] = - Mt[0:3, 0:3] @ M[0:3, 3]
    Mt[3, 3] = 1
    return Mt

my_inv = inv_rigid(view_matrix)
inv = np.linalg.inv(view_matrix)
print(np.linalg.det(view_matrix))
print(np.linalg.norm(my_inv - inv))
print(np.abs(my_inv - inv))
print(my_inv)
print(inv)
